In [24]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import numpy as np
import csv
import os
# from scipy.fft import fft
# %matplotlib

In [2]:
def fft_window(xyz_win, fs=25):

    fft_x=fft(xyz_win[:,0])
    fft_y=fft(xyz_win[:,1])
    fft_z=fft(xyz_win[:,2])
    freq_x=np.fft.fftfreq(len(xyz_win[:,0]))*fs
    power_x=np.abs(fft_x)**2
    freq_y=np.fft.fftfreq(len(xyz_win[:,1]))*fs
    power_y=np.abs(fft_y)**2
    freq_z=np.fft.fftfreq(len(xyz_win[:,2]))*fs
    power_z=np.abs(fft_z)**2

    max_freq_x_indices=np.argmax(power_x)
    main_freq_x=freq_x[max_freq_x_indices]
    max_freq_y_indices=np.argmax(power_y)
    main_freq_y=freq_y[max_freq_y_indices]
    max_freq_z_indices=np.argmax(power_z)
    main_freq_z=freq_z[max_freq_z_indices]
            
    return main_freq_x,main_freq_y,main_freq_z

def func_calculation(data, func_size, func):
    data_len = len(data)
    half_len = int((func_size-1)/2)
    func_data = np.zeros(data_len)
    for i in range(data_len):
        if i <= half_len:
            func_data[i] = func(data[0:2*i+1])
        else:
            if (i + half_len < data_len):
                func_data[i] = func(data[(i - half_len):(i + half_len+1)])
            else:
                func_data[i] = func(data[(2*i+1-data_len):data_len])
    return func_data

def remove_mean(a):
    return a-np.mean(a)


def if_a_in_B(a, B):
    B_len = len(B)
    i = 0
    r = 0
    while i < B_len:
        if B[i] == a:
            r = 1
            break
        i = i + 1
    return r


def delete_ith_a(a, i):
    a = np.array(a)
    a_len = len(a)
    if a_len <= 1 and i == 0:
        return np.zeros(0)
    elif i < 0 or i > a_len - 1:
        return a
    else:
        if i == 0:
            return a[1:a_len]
        elif i == a_len-1:
            return a[0:a_len-1]
        else:
            return np.concatenate((a[0:i], a[i+1:a_len]))


class action_processor:
    def __init__(self, peak_num=125, valley_num=125):
        self.p_loc = np.zeros(peak_num, dtype=int)
        self.p_cnt = int(0)
        self.v_loc = np.zeros(valley_num, dtype=int)
        self.v_cnt = int(0)

    def find_possible_peak_valley(self, a):  
        a_len = len(a)
        for i in range(1, a_len-1):   
            if a[i] >= a[i-1] and a[i] > a[i+1]: 
                self.p_loc[self.p_cnt] = i
                self.p_cnt = self.p_cnt + 1
            if a[i] <= a[i-1] and a[i] < a[i+1]:
                self.v_loc[self.v_cnt] = i
                self.v_cnt = self.v_cnt + 1
        self.p_loc = self.p_loc[0:self.p_cnt]
        self.v_loc = self.v_loc[0:self.v_cnt]
        return self

    def merge_close_pole(self, a, direction): 
        if direction == 1:                
            pole_locs = self.p_loc
            pole_cnt = self.p_cnt
            v_loc = self.v_loc
        elif direction == -1:                
            pole_locs = self.v_loc
            pole_cnt = self.v_cnt
            v_loc = self.p_loc
        if pole_cnt > 1:    
            i = 1
            while i < pole_cnt:   
                r = 0
                j = pole_locs[i-1]       
                while j < pole_locs[i]:        
                    r = if_a_in_B(j, v_loc)   
                    if r == 1:                 
                        break
                    j = j + 1
                if r == 0:                     
                    if a[pole_locs[i-1]]*direction > a[pole_locs[i]]*direction:   
                        pole_locs = delete_ith_a(pole_locs, i)
                    else:
                        pole_locs = delete_ith_a(pole_locs, i-1)
                    pole_cnt = pole_cnt - 1
                else:
                    i = i + 1

        if direction == 1:      
            self.p_loc = pole_locs
            self.p_cnt = pole_cnt
        else:
            self.v_loc = pole_locs
            self.v_cnt = pole_cnt
        return self

    def merge_close_peaks_valleys(self, a): 
        if self.p_cnt > 1:                 
            self.merge_close_pole(a, 1)
        if self.v_cnt > 1:                 
            self.merge_close_pole(a, -1)
        if self.p_cnt > 0 and ((self.v_cnt > 0 and self.p_loc[0] < self.v_loc[0]) or
                               (self.v_cnt == 0)):
            self.p_loc = delete_ith_a(self.p_loc, 0)
            self.p_cnt = self.p_cnt - 1
        return self

    def remove_false_peak_valley(self, a): 
        a_mean = np.mean(a)
        i = 0
        while i < self.p_cnt:
            if a[self.p_loc[i]] < a_mean:       
                self.p_loc = delete_ith_a(self.p_loc, i)
                self.p_cnt = self.p_cnt - 1
            else:
                i = i + 1
        i = 0
        while i < self.v_cnt:
            if a[self.v_loc[i]] > a_mean:        
                self.v_loc = delete_ith_a(self.v_loc, i)
                self.v_cnt = self.v_cnt - 1
            else:
                i = i + 1

        return self

    def remove_asymmetric_peaks(self, a): 
        last_valley_loc = int(0)
        TIME_THRESHOLD1 = 4             
        TIME_THRESHOLD2 = 40
        self.h1=[]
        self.h2=[]
        self.t1=[]
        self.t2=[]
        if self.v_cnt <= 1:             
            return self
        else:
            i = 0
            while i < self.p_cnt:
                r1 = 0
                r2 = 0
                j = 1  # 波谷
                while j < self.v_cnt:   
                    if self.v_loc[j-1] < self.p_loc[i] and self.v_loc[j] > self.p_loc[i]:   
                        r1 = 1  # 记录
                        h1 = np.abs(a[self.p_loc[i]] - a[self.v_loc[j-1]]) 
                        self.h1.append(h1)
                        h2 = np.abs(a[self.p_loc[i]] - a[self.v_loc[j]])
                        self.h2.append(h2)
                        t1 = np.abs(self.p_loc[i] - self.v_loc[j-1])
                        self.t1.append(t1)
                        t2 = np.abs(self.p_loc[i] - self.v_loc[j])
                        self.t2.append(t2)
                        if h1 > PEAK_VALLEY_DIFFERENCE and h2 > PEAK_VALLEY_DIFFERENCE and \
                            h1 > h2/2 and h1 < h2*2 and \
                                t1 >= TIME_THRESHOLD1 and t1 <= TIME_THRESHOLD2 and t2 >= TIME_THRESHOLD1 and t2 <= TIME_THRESHOLD2:
                            last_valley_loc = self.v_loc[j]
                        else:     # 否则删除这个波峰
                            r2 = 1
                            self.p_loc = delete_ith_a(self.p_loc, i)
                            self.p_cnt = self.p_cnt - 1
                            if last_valley_loc != self.v_loc[j-1]:
                                self.v_loc = delete_ith_a(self.v_loc, j-1)
                                self.v_cnt = self.v_cnt - 1
                        break
                    j = j + 1
                if r1 == 0:
                    self.p_loc = delete_ith_a(self.p_loc, i)
                    self.p_cnt = self.p_cnt - 1
                elif r2 == 0:
                    i = i + 1

        return self



In [3]:
import os

file = r'IMU_Dataset'
file_list=[]
for root, dirs, files in os.walk(file):
    for file in files:
        path = os.path.join(root, file)
        # print(path)
        file_list.append(path)


In [4]:
file_list

['IMU_Dataset\\ReadMe.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-05-11-15_37_01_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-05-11-16_40_53_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-05-11-18_05_41_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-09_45_23_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-10_13_23_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-10_56_32_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-11_30_36_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-14_32_45_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-15_19_03_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-15_45_22_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-16_35_00_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-16_49_42_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-2023-06-01-17_21_40_good_morning.txt',
 'IMU_Dataset\\good_morning\\IMU-202

In [5]:
# preprocessing data
data_list=[]
label_list=[]
peak_list=[0,0,0,0]
array_size=8
data_array = np.empty((0, 19 + 2))  

if __name__ == "__main__":
    for file in file_list[1:]:
        # print(file)
        data_len = 100000
        data = np.zeros((data_len,array_size))
        cnt = 0
        for line in open(file):
            cur_data=line.split(",")
            for i in range(6):
                data[cnt][i] = cur_data[i]
            cnt = cnt + 1
        # 去除头尾信号
        data = data[0:cnt]
        data_len = cnt
        step_total = 0
        acc_len=cnt
        acc_data=data
        
        m1 = 11
        m2 = 5
        acc_xyz_mean = np.zeros((acc_len, array_size))
        acc_xyz_lable= np.zeros((acc_len,1))
        for i in range(6):         # 11 and 5 smooth
            tmp_data = acc_data[:, i]  
            tmp_data = func_calculation(tmp_data, m1, np.mean)
            tmp_data = tmp_data.astype(int)
            tmp_data = func_calculation(tmp_data, m2, np.mean)
            tmp_data = tmp_data.astype(int)
            acc_xyz_mean[:, i] = tmp_data
        acc_xyz_mean[:, 6] = ((acc_xyz_mean[:, 0]/4)**2 + (acc_xyz_mean[:, 1]/4)**2 + (acc_xyz_mean[:, 2]/4)**2)
        acc_xyz_mean[:, 7] = ((acc_xyz_mean[:, 3]/4)**2 + (acc_xyz_mean[:, 4]/4)**2 + (acc_xyz_mean[:, 5]/4)**2)

        fs = 25        
        win_sec = 5   
        buf_sec = 3  
        win_len = win_sec*fs
        buf_len = buf_sec*fs
        xyz_win = np.zeros((win_len,array_size)) 
        win_cnt = int(0)
        xyz_buf = np.zeros((buf_len,array_size))
        buf_cnt = np.zeros(array_size,dtype=int)   
        ACC_SENSOR_GRAVITY = 4096        
        STEP_ACC_DIFF_THRESHOLD = 4096 // 10   
        PEAK_VALLEY_DIFFERENCE = 4096 // 14    
        LEFT_DATA_NUM = int(2)

        step_total = 0
        axis_label = ['x','y','z']
        delay_point = fs - m1//2 - m2//2     

        for i in range(fs+delay_point,acc_len,fs):
            # pick one second data 
            xyz_sec = acc_xyz_mean[(i-fs):i,0:array_size]    # start:(i-fs) end:(i)  for every element, pick[0:3], so at last, shape:fs*3
            if win_cnt + fs <= win_len:             # not fill the window
                xyz_win[win_cnt:(win_cnt+fs), :] = xyz_sec
                win_cnt = win_cnt + fs
            if win_cnt >= win_len:                 
                p_loc=[]
                v_loc=[]
                p_cnt=[]
                v_cnt=[]
                for j in range(array_size):
                    if buf_cnt[j] > 0:
                        axis_win = np.concatenate((xyz_buf[0:buf_cnt[j], j], xyz_win[:, j]))
                        axis_len = win_cnt + buf_cnt[j]  
                    else:
                        axis_win = xyz_win[:, j]  
                        axis_len = win_cnt
                    xyz_buf[:, j] = np.zeros(buf_len)  
                    buf_cnt[j] = int(0)

                    tmp_max = np.max(axis_win, 0)     
                    tmp_min = np.min(axis_win, 0)    
                    tmp_diff = np.max(tmp_max - tmp_min) 
                    axis_process = action_processor(125, 125)  
                    if tmp_diff > STEP_ACC_DIFF_THRESHOLD:  
                        axis_process = axis_process.find_possible_peak_valley(axis_win)  

                        axis_process = axis_process.remove_false_peak_valley(axis_win) 

                        axis_process = axis_process.merge_close_peaks_valleys(axis_win)  
                        
                        axis_process = axis_process.remove_asymmetric_peaks(axis_win)    
                        if axis_process.v_cnt >= 1:
                            last_v_loc = axis_process.v_loc[axis_process.v_cnt-1]
                            # peak_list[j]-=1
                            left_len = axis_len - int(last_v_loc) + LEFT_DATA_NUM
                            if left_len < buf_len:
                                xyz_buf[0:left_len, j] = axis_win[(axis_len-left_len):axis_len]
                                buf_cnt[j] = int(left_len)
                        p_loc.append(axis_process.p_loc)
                        v_loc.append(axis_process.v_loc)
                        p_cnt.append(axis_process.p_cnt)
                        v_cnt.append(axis_process.v_cnt)
                        
                mean_gyro_x = np.mean(xyz_win[:,6])
                mean_acc_x = np.mean(xyz_win[:,7])
                
                variance_gyro_x = np.var(xyz_win[:,0])
                variance_gyro_y = np.var(xyz_win[:,1])
                variance_gyro_z = np.var(xyz_win[:,2])

                variance_acc_x = np.var(xyz_win[:,3])
                variance_acc_y = np.var(xyz_win[:,4])
                variance_acc_z = np.var(xyz_win[:,5])

                covariance_gyro_matrix = np.cov(np.array([xyz_win[:,0], xyz_win[:,1], xyz_win[:,2]]))
                covariance_acc_matrix = np.cov(np.array([xyz_win[:,3], xyz_win[:,4], xyz_win[:,5]]))

                covariance_gyro_xy = np.abs(covariance_gyro_matrix[0, 1])
                covariance_gyro_xz = np.abs(covariance_gyro_matrix[0, 2])
                covariance_gyro_yz = np.abs(covariance_gyro_matrix[1, 2])

                covariance_acc_xy = np.abs(covariance_acc_matrix[0, 1])
                covariance_acc_xz = np.abs(covariance_acc_matrix[0, 2])
                covariance_acc_yz = np.abs(covariance_acc_matrix[1, 2])

                diff_gyro_x = np.ptp(xyz_win[:,0])
                diff_gyro_y = np.ptp(xyz_win[:,1])
                diff_gyro_z = np.ptp(xyz_win[:,2])

                diff_acc_x = np.ptp(xyz_win[:,3])
                diff_acc_y = np.ptp(xyz_win[:,4])
                diff_acc_z = np.ptp(xyz_win[:,5])
                
                if len(p_loc)<6:
                    lambda_gyro_x=0
                    lambda_gyro_y=0
                    lambda_gyro_z=0
                    lambda_acc_x=0
                    lambda_acc_y=0
                    lambda_acc_z=0
                else:
                    if len(p_loc[0])>1:
                        arr_gyro_x0=p_loc[0][:-1]
                        arr_gyro_x1=p_loc[0][1:]
                        lambda_gyro_x = np.mean(arr_gyro_x1-arr_gyro_x0)
                    else:
                        lambda_gyro_x=0
                    if len(p_loc[1])>1:
                        arr_gyro_y0=p_loc[1][:-1]
                        arr_gyro_y1=p_loc[1][1:]
                        lambda_gyro_y = np.mean(arr_gyro_y1-arr_gyro_y0)
                    else:
                        lambda_gyro_y=0
                    if len(p_loc[2])>1:
                        arr_gyro_z0=p_loc[2][:-1]
                        arr_gyro_z1=p_loc[2][1:]
                        lambda_gyro_z = np.mean(arr_gyro_z1-arr_gyro_z0)
                    else:
                        lambda_gyro_z=0
                    if len(p_loc[3])>1:
                        arr_acc_x0=p_loc[3][:-1]
                        arr_acc_x1=p_loc[3][1:]
                        lambda_acc_x = np.mean(arr_acc_x1-arr_acc_x0)
                    else:
                        lambda_acc_x=0
                    if len(p_loc[4])>1:
                        arr_acc_y0=p_loc[4][:-1]
                        arr_acc_y1=p_loc[4][1:]
                        lambda_acc_y = np.mean(arr_acc_y1-arr_acc_y0)
                    else:
                        lambda_acc_y=0
                    if len(p_loc[5])>1:
                        arr_acc_z0=p_loc[5][:-1]
                        arr_acc_z1=p_loc[5][1:]
                        lambda_acc_z = np.mean(arr_acc_z1-arr_acc_z0)
                    else:
                        lambda_acc_z=0
                        
                if len(p_cnt)==8:
                    freq_gyro_x = p_cnt[0]
                    freq_gyro_y = p_cnt[1]
                    freq_gyro_z = p_cnt[2]
                    freq_acc_x = p_cnt[3]
                    freq_acc_y = p_cnt[4]
                    freq_acc_z = p_cnt[5]
                else:
                    freq_gyro_x = 0
                    freq_gyro_y = 0
                    freq_gyro_z = 0
                    freq_acc_x = 0
                    freq_acc_y = 0
                    freq_acc_z = 0
                
                # result_array = np.array([mean_gyro_x, variance_gyro_x, variance_gyro_y, variance_gyro_z, covariance_gyro_xy, covariance_gyro_xz, covariance_gyro_yz,
                #                          lambda_gyro_x, lambda_gyro_y, lambda_gyro_z,
                #                          mean_acc_x, variance_acc_x, variance_acc_y, variance_acc_z, covariance_acc_xy, covariance_acc_xz, 
                #                          covariance_acc_yz, lambda_acc_x, lambda_acc_y, lambda_acc_z])
                result_array = np.array([mean_gyro_x, variance_gyro_x, variance_gyro_y, variance_gyro_z, 
                                         covariance_gyro_xy, covariance_gyro_xz, covariance_gyro_yz,
                         freq_gyro_x, freq_gyro_y, freq_gyro_z,
                         mean_acc_x, 
                                         variance_acc_x, variance_acc_y, variance_acc_z, 
                                         covariance_acc_xy, covariance_acc_xz,covariance_acc_yz, 
                                         freq_acc_x, freq_acc_y, freq_acc_z])
                xyz_win = np.zeros((win_len, array_size))
                win_cnt = int(0)
                #   IMU_ACT_NONE, /**< None action */
                # 0 IMU_ACT_JUMPING_SQUAT, /**< Jumping squat action */ 
                # 1 IMU_ACT_JUMPING_JACK, /**< Jumping jack action */
                # 2 IMU_ACT_JUMPING_LUNGE, /**< Jumping lunge action */
                # 3 IMU_ACT_SIT, /**< Sit action */
                # 4 IMU_ACT_SQUAT, /**< Squat action */
                # 5 IMU_ACT_TROT, /**< Trot action */
                # 6 IMU_ACT_WALK, /**< Walk action */
                # 7 IMU_ACT_WAVE, /**< Wave action */
                # 8 IMU_ACT_LUNGE, /**< Lunge action */
                # 9 IMU_ACT_GOOD_MORNING, /**< Good morning action */
                # 10 IMU_ACT_TUCK_JUMPING, /**< Tuck jumping action */
                if ("sit" in file):
                    label=4
                elif ("walk" in file):
                    label=3
                elif ("trot" in file):
                    label=3
                elif ("wave" in file):
                    label=3
                elif ("good_morning" in file):
                    label=3
                elif ("jumping_jack" in file):
                    label=1
                elif ("jumping_lunge" in file):
                    label=2
                elif ("jumping_squat" in file):
                    label=0
                elif ("tuck_jump" in file):
                    label=3
                elif ("squat" in file):
                    label=3
                elif ("lunge" in file):
                    label=3
                labeled_array = np.append(result_array, label)
                data_array = np.vstack((data_array, labeled_array))
                
                # print(freq_gyro_x, freq_gyro_y, freq_gyro_z,freq_acc_x, freq_acc_y, freq_acc_z)


In [14]:
def save_list_to_csv(data_list, file_path):
    with open(file_path,'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in data_list:
            writer.writerow(row)

In [15]:
# np.set_printoptions(suppress=True)
# print(list(data_array))

# file_path = 'feature.csv'
# save_list_to_csv(list(data_array),file_path)

[array([ 155023.3425    ,  174359.780224  , 1897872.179456  ,
        404364.705536  ,  563529.01664516,  254092.29535484,
        858075.27187097,       1.        ,       1.        ,
             1.        , 1151436.627     , 6479623.688064  ,
       1751098.447104  , 1726270.578816  , 3319715.19477419,
       3177914.25890323, 1662349.20141936,       2.        ,
             1.        ,       1.        ,       3.        ]), array([ 211729.4025    ,  234247.325824  , 2281779.3664    ,
        843038.774016  ,  692155.13967742,  409934.75496774,
       1386205.24225806,       2.        ,       2.        ,
             3.        , 1257316.924     , 4800618.843264  ,
       2884537.0496    , 2509536.424576  , 3609562.28096774,
       3409995.85787097, 2697762.58451613,       2.        ,
             3.        ,       3.        ,       3.        ]), array([ 138422.2865    ,  211415.422976  , 1444371.382016  ,
        535792.437376  ,  536314.28296774,  325102.23167742,
        878828.2363

In [19]:
a=np.array([7.327571,19.738663,13.597322,-0.859467,20.835178,12.093331,7.652948,-1.696852,-2.931637,-1.674842,5.074912,2.353767,13.183642,1.252337,10.654410,-4.463031,25.668964,-3.610969,12.193342,10.442342])
np.var(a)

73.69573458707714

In [20]:
tmp_list=[0]*11
total=0
for i in data_array:
    # print(i[-1])
    tmp_list[int(i[-1])]+=1

print(tmp_list)
for i in tmp_list:
    total+=i
    
print(total)

[253, 356, 247, 2346, 510, 0, 0, 0, 0, 0, 0]
3712


In [21]:
data_array[-1]
data_list=data_array[:,0:-1]
label_list=data_array[:,-1:]

In [22]:
X_train_list=np.array(data_list)
X_train=X_train_list
y_train_list=np.array(label_list)
y_train=y_train_list[:,0]

In [23]:
X_train_list

array([[ 155023.3425  ,  174359.780224, 1897872.179456, ...,
              2.      ,       1.      ,       1.      ],
       [ 211729.4025  ,  234247.325824, 2281779.3664  , ...,
              2.      ,       3.      ,       3.      ],
       [ 138422.2865  ,  211415.422976, 1444371.382016, ...,
              3.      ,       2.      ,       2.      ],
       ...,
       [ 218007.1335  ,  754368.502784,  409969.296256, ...,
              3.      ,       4.      ,       3.      ],
       [ 274644.8775  , 1159979.171584,  424641.011456, ...,
              1.      ,       3.      ,       4.      ],
       [ 227055.5825  ,  885147.592064,  206954.481536, ...,
              1.      ,       4.      ,       3.      ]])

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from tqdm import tqdm
# from model.attention.SelfAttention import ScaledDotProductAttention
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from joblib import dump, load
import copy
print(torch.__version__)
# 随机数种子， 方便复现
torch.manual_seed(0)
device=torch.device("cpu")
# device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark=True
# print(torch.cuda.get_device_name(0))

1.13.1+cpu


In [4]:
# class SCModel(nn.Module):
#     def __init__(self, in_channels=32, num_classes=5):
#         super(SCModel, self).__init__()
#         self.relu = nn.ReLU()
#         self.linear1 = nn.Linear(20,128)
#         self.batch = nn.BatchNorm1d(128)
#         self.batch2 = nn.BatchNorm1d(64)
#         # self.pool = nn.MaxPool1d(10)
#         self.linear2 = nn.Linear(128, 64)
#         self.linear3 = nn.Linear(64, num_classes)
#         self.soft = nn.Softmax(dim=1)
    

#     def forward(self, x):
#         x = self.linear1(x) 
#         x = self.batch(x)  
#         x = self.relu(x) 
#         x = self.linear2(x)
#         x = self.batch2(x)  
#         x = self.relu(x)
#         x=self.linear3(x)
#         x = self.soft(x)
#         # print(x)
#         return x
class SCModel(nn.Module):
    def __init__(self):
        super(SCModel, self).__init__()
        self.linear1 = nn.Linear(20,64)
        self.batch = nn.BatchNorm1d(20)
        self.linear2 = nn.Linear(64, 20)
        self.pool = nn.MaxPool1d(4)
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim=1)
    

    def forward(self, x):
        x = self.batch(x)  
        x = self.linear1(x) 
        x = self.relu(x) 
        x = self.linear2(x)
        x = self.pool(x) 
        x = self.soft(x)
        return x

In [5]:
model=SCModel()
model=model.to(device)
model._modules
model.double()

SCModel(
  (linear1): Linear(in_features=20, out_features=64, bias=True)
  (batch): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=64, out_features=20, bias=True)
  (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (soft): Softmax(dim=1)
)

In [114]:
summary(model)

Layer (type:depth-idx)                   Param #
SCModel                                  --
├─Linear: 1-1                            1,344
├─BatchNorm1d: 1-2                       40
├─Linear: 1-3                            1,300
├─MaxPool1d: 1-4                         --
├─ReLU: 1-5                              --
├─Softmax: 1-6                           --
Total params: 2,684
Trainable params: 2,684
Non-trainable params: 0

In [115]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)
x_train_torch=torch.from_numpy(X_train)
y_train_torch=torch.from_numpy(y_train)

train_dataset = TensorDataset(x_train_torch,y_train_torch)
split_tra=int(len(y_train)*0.7)
split_tes=len(y_train)-split_tra
train_loader,test_loader=random_split(dataset=train_dataset,lengths=[split_tra,split_tes])
train_loader=DataLoader(dataset=train_loader,batch_size=16,shuffle=True,drop_last=True)
# train_loader=DataLoader(dataset=train_loader,batch_size=32,shuffle=False,drop_last=True,pin_memory=False)
test_loader=DataLoader(dataset=test_loader,batch_size=16,shuffle=True,drop_last=True)

In [116]:
best_acc=0.0
for i in range(6):
# epochs=6
    lr_s=5e-3
    optimizer=torch.optim.Adamax(model.parameters(),lr=lr_s)
    epochs=150
    acc_list=[]
    loss_list=[]
    best_model=model
    # kinds=2
    # best_teacher_matrix=torch.zeros(kinds, kinds)
    for epoch in range(epochs):
        if (epoch%5==0):
            lr_s=lr_s-0.1*lr_s
            if(lr_s<1e-6):
                lr_s*=10
            print("lr=",lr_s)
        optimizer=torch.optim.Adam(model.parameters(),lr=lr_s)
        model.train()
        total_loss=0.0
        #训练集上训练模型权重
        for data,targets in tqdm(train_loader):
            data=data.to(device)
            targets=targets.to(device)
            #前向预测
            preds=model(data)
            loss=criterion(preds,targets.long())+0.2
            total_loss+=loss.item()
            
            #反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # print("loss:",total_loss/len(train_loader))
        loss_list.append(total_loss/len(train_loader))
        # 测试集上评估模型性能
        model.eval()
        num_correct=0
        num_samples=0
        with torch.no_grad():
            for x,y in test_loader:
                x=x.to(device)
                y=y.to(device)

                preds=model(x)
                # conf_matrix = confusion_matrix(preds, y, conf_matrix)
                predictions=preds.max(1).indices
                num_correct+=(predictions==y).sum()
                num_samples+=predictions.size(0)
            acc=(num_correct/num_samples).item()
            if(acc>best_acc):
                best_teacher_model=copy.deepcopy(model)
                best_acc=acc
                # best_teacher_matrix=copy.deepcopy(conf_matrix)
                # torch.save(model, 'best_teacher_model.pth')
            print(num_correct,num_samples)
        print("Epoch:{}\t Accuracy:{:.4f}".format(epoch+1,acc))
        acc_list.append(acc)
        model.train()


100%|███████████████████████████████████████████████████████████████████████████████| 162/162 [00:00<00:00, 830.76it/s]

lr= 0.0045000000000000005



 53%|██████████████████████████████████████████▍                                     | 86/162 [00:00<00:00, 843.15it/s]

tensor(897) 1104
Epoch:1	 Accuracy:0.8125


100%|███████████████████████████████████████████████████████████████████████████████| 162/162 [00:00<00:00, 826.53it/s]

...


 54%|███████████████████████████████████████████▍                                    | 88/162 [00:00<00:00, 873.82it/s]

tensor(945) 1104
Epoch:8	 Accuracy:0.8560


  0%|                                                                                          | 0/162 [00:00<?, ?it/s]

tensor(982) 1104
Epoch:9	 Accuracy:0.8895


KeyboardInterrupt: 

In [69]:
best_acc

0.948369562625885

In [70]:
new_model=model

In [71]:
new_model=copy.deepcopy(best_model)

In [72]:
torch.save(new_model, 'best_model_final_5class_94836.pth')

In [15]:
new_model=torch.load('best_model_final_5class_94836.pth')

In [16]:
new_model.eval()

SCModel(
  (linear1): Linear(in_features=20, out_features=64, bias=True)
  (batch): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=64, out_features=20, bias=True)
  (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (soft): Softmax(dim=1)
)

In [7]:
new_model.batch.weight

Parameter containing:
tensor([1.4630, 1.1395, 1.2177, 1.9351, 1.4001, 1.4318, 1.2889, 1.8409, 1.5705,
        1.5407, 2.0648, 1.7423, 2.1437, 1.9154, 1.9112, 1.4830, 1.1438, 1.4302,
        1.4221, 1.2019], dtype=torch.float64, requires_grad=True)

In [74]:
new_model.batch.bias

Parameter containing:
tensor([ 0.4149,  0.2515,  0.4687, -0.0728, -0.0123,  0.1118,  0.2080,  0.2811,
         0.2876,  0.2211,  0.1636,  0.2874,  0.1514,  0.2083,  0.2321,  0.7242,
         0.4849,  0.5368,  0.3365,  0.5430], dtype=torch.float64,
       requires_grad=True)

In [90]:
# for i in new_model.batch.running_var:
#     print(str(i)+',',end='')
new_model.batch.running_var

tensor([9.2564e+10, 1.9401e+12, 1.0174e+12, 1.4391e+13, 4.4327e+11, 2.0081e+12,
        8.4632e+11, 4.3679e+00, 4.2292e+00, 4.9381e+00, 1.7119e+12, 1.2815e+13,
        1.5395e+13, 1.9554e+12, 5.6540e+12, 1.9520e+12, 1.7465e+12, 5.7742e+00,
        5.3014e+00, 5.1225e+00])

In [88]:
new_model.batch.running_mean

tensor([2.0781e+05, 7.9986e+05, 5.7581e+05, 1.8422e+06, 4.0829e+05, 7.8570e+05,
        5.5670e+05, 2.6223e+00, 2.6036e+00, 2.7710e+00, 1.5957e+06, 1.8603e+06,
        1.9816e+06, 1.0489e+06, 1.2904e+06, 8.3088e+05, 8.2307e+05, 2.8332e+00,
        2.7074e+00, 2.7247e+00])

In [89]:
new_model.linear1.bias

Parameter containing:
tensor([ 2.0328e-01,  2.0152e-01, -1.0325e-01,  5.0100e-01, -2.6685e-01,
        -1.1462e-01,  3.7187e-01,  5.9041e-01, -3.8752e-01, -1.2642e-01,
         6.2581e-02, -4.9769e-01, -3.8052e-01,  5.0055e-01, -6.9461e-01,
         5.8588e-01,  8.7461e-02,  1.1136e-01, -1.6738e-01, -1.9445e-01,
        -4.3587e-01, -5.4425e-01,  6.0102e-01, -1.0060e-01,  6.5055e-01,
         5.7548e-01, -5.7315e-01, -3.5801e-01, -3.1732e-01,  2.3966e-01,
         3.8924e-04, -4.9888e-01,  4.7820e-02, -5.4354e-01,  7.1393e-01,
        -8.2805e-02,  4.7776e-01, -1.4557e-01,  6.6176e-01, -2.6903e-01,
         2.8295e-01,  4.4839e-01, -9.0092e-02,  2.2224e-01, -4.9842e-01,
         2.2000e-01,  6.2266e-01,  5.3258e-02,  1.5899e-01,  2.5958e-01,
         1.3770e-01, -4.0305e-01,  3.2315e-01,  1.4999e-01,  3.8500e-01,
        -6.2552e-02,  4.9460e-01, -1.6007e-01,  1.3765e-01,  5.0103e-01,
        -5.3695e-02,  3.3068e-01,  3.8185e-01,  5.6207e-01],
       requires_grad=True)

In [78]:
len(new_model.linear1.bias)

64

In [79]:
for i in new_model.linear1.weight:
    for j in i:
        print(str(float(j))+', ',end="")

-0.4825854629403204, 0.07449666251875195, -0.5777879502629854, 0.5784153720937505, -0.1537514694900409, 0.15311214468768589, 0.02377098597881867, 1.038303212253491, -0.0731959926199297, 1.228498662677599, -0.05576929055667945, -0.305622268895184, -0.3830046052096684, -0.36129645948174033, 0.17093496880112766, -0.8660672945330196, -0.6024525429059828, 0.4301694494612756, -0.6844211738893194, -0.56745313326157, -0.001125190051249332, 0.4546547455989288, 0.3127684685377651, -0.02078444990033989, 0.13536513065831904, -0.15010528390871405, 0.17119712606708257, -1.375404194307566, 0.3469116173411126, -0.43653362703266996, 0.38739208539778086, 0.5527105880721837, 0.23783317377934912, 0.3915913233869221, -0.7520509628093229, -0.9334742608557521, -0.22252163977728265, 0.8242041158573248, 0.4595236093826225, 1.31123368387263, 0.0539379184771505, -0.5668012152359413, 0.1355399451389175, -0.4712707928292526, 0.0760856529498568, -0.03220133873977195, 0.050518765322386444, 0.08073613731946623, -0.80

In [80]:
new_model.linear2.bias

Parameter containing:
tensor([-0.2212, -0.7254, -0.5962, -0.7691, -0.8325, -0.7905, -0.8129,  0.1621,
        -0.7333, -0.6498, -0.5797,  0.4608, -0.1686, -0.0107, -0.1093, -0.0129,
        -0.8300, -0.8633, -0.8028, -0.3941], dtype=torch.float64,
       requires_grad=True)

In [81]:
for i in new_model.linear2.weight:
    for j in i:
        print(str(float(j))+', ',end="")

-2.6846286279779012, 0.5825917654362386, 0.691406353529651, -0.4813607227059867, -0.2886242766496942, -0.33302626005920144, -0.503705450466225, 0.15186091929443904, -2.20956351789154, 0.5091251566587538, 0.5358421294659358, 0.2589227586307499, -2.03559623083287, -0.12211803576100555, -2.382771433754252, -2.0641097160608037, 0.7157480017491489, 0.23955098563126634, -1.1571640862088415, 0.3066317965776676, 0.6290802265742907, -2.2870266884896497, -1.08113448944131, -1.7607167963707218, -2.257319088557373, 0.32288147796607286, -2.5098149937613616, 0.6278481365229909, -0.16056676617897056, -1.4159122989116315, -1.2925812135672634, 0.42682990725579645, 0.5029434981021774, 0.25196574455099807, 0.23279628298714145, -0.4499763350331512, 1.012098840457713, -0.8259155072925083, -0.598257402857946, -1.6330442766177966, 1.035147407368816, 0.32380051074981886, 0.40992561739879435, 0.18530652803084513, -0.7226132984538558, 0.16130926459603273, -0.12627759867899407, -0.9257166163898712, 0.71162158465

In [82]:
len(new_model.linear2.weight[0])

64

In [45]:
# x_train_torch=torch.from_numpy(X_train)
# y_train_torch=torch.from_numpy(y_train)

# train_dataset = TensorDataset(x_train_torch,y_train_torch)
# split_tra=int(len(y_train))
# train_loader=DataLoader(dataset=train_dataset,batch_size=16,shuffle=False,drop_last=True)
# # 测试集上评估模型性能
# new_model.eval()
# num_correct=0
# num_samples=0
# with torch.no_grad():
#     for x,y in train_loader:
#         # print(x)
#         x=x.to(device)
#         y=y.to(device)
#         preds=new_model(x)
#         predictions=preds.max(1).indices
#         # print(predictions,y)
#         num_correct+=(predictions==y).sum()
#         num_samples+=predictions.size(0)
#     acc=(num_correct/num_samples).item()
#     print(num_correct,num_samples)
    
# compare
test=[155023.3425, 174359.7802, 1897872.179, 404364.7055, 563529.0166, 254092.2954, 858075.2719, 1.0, 1.0, 1.0, 1151436.627, 6479623.688, 1751098.447, 1726270.579, 3319715.195, 3177914.259, 1662349.201, 2.0, 1.0, 1.0]
test_torch=torch.tensor(test).unsqueeze(0).float()
# new_model = new_model.float()
batch=new_model.batch(test_torch)
print(batch)
linnear1_out=new_model.linear1(batch)
rel=new_model.relu(linnear1_out)
print(rel)
linnear2_out=new_model.linear2(rel)
print(linnear2_out)
pool=new_model.pool(linnear2_out)
print(pool)
# output = new_model(test_torch)
# print(output)

tensor([[ 0.1610, -0.2602,  2.0647, -0.8062,  0.3142, -0.4253,  0.6302, -1.1479,
         -0.9370, -1.0068, -0.5375,  2.5357,  0.0255,  1.1361,  1.8631,  3.2155,
          1.2113,  0.0409, -0.7180, -0.3728]],
       grad_fn=<NativeBatchNormBackward0>)
tensor([[0.0000, 0.0000, 7.1167, 4.5024, 6.3679, 0.0000, 3.5684, 4.2245, 0.0000,
         0.3667, 0.0000, 0.0000, 0.0000, 2.5746, 0.0000, 0.0000, 0.0000, 8.3424,
         0.0000, 0.0000, 0.0000, 0.0000, 2.3632, 2.7008, 0.0000, 3.7570, 0.0000,
         0.0000, 0.0000, 0.0000, 5.5283, 2.1828, 0.0000, 3.7811, 1.1618, 0.7538,
         6.0943, 0.0000, 0.0000, 2.9114, 0.0000, 2.7774, 0.0787, 8.1459, 0.0000,
         0.5774, 6.4544, 5.2262, 6.3719, 7.4002, 6.7245, 3.1117, 1.0825, 0.0000,
         5.8387, 0.0000, 2.7359, 0.0000, 4.2991, 3.8957, 0.0000, 0.9025, 1.1191,
         0.0000]], grad_fn=<ReluBackward0>)
tensor([[ -12.6277,  -38.2584,  -41.4612,  -38.5054,  -54.1381,  -50.6980,
          -52.9273,  -72.6814,  -57.1774,  -59.3092,  -53.3891

In [29]:
file = os.path.join(os.getcwd()+'\\MyNN', 'feature.csv')
test_set=[]
with open (file,"r") as f:
    reader = csv.reader(f)
    for row in reader:
        test_set.append(list(map(float, row)))
        

In [38]:
print(test_set)

[[155023.3425, 174359.7802, 1897872.179, 404364.7055, 563529.0166, 254092.2954, 858075.2719, 1.0, 1.0, 1.0, 1151436.627, 6479623.688, 1751098.447, 1726270.579, 3319715.195, 3177914.259, 1662349.201, 2.0, 1.0, 1.0], [211729.4025, 234247.3258, 2281779.366, 843038.774, 692155.1397, 409934.755, 1386205.242, 2.0, 2.0, 3.0, 1257316.924, 4800618.843, 2884537.05, 2509536.425, 3609562.281, 3409995.858, 2697762.585, 2.0, 3.0, 3.0], [138422.2865, 211415.423, 1444371.382, 535792.4374, 536314.283, 325102.2317, 878828.2364, 3.0, 3.0, 2.0, 1105435.031, 3200831.105, 2409240.117, 1871791.449, 2708281.313, 2396986.33, 2124174.866, 3.0, 2.0, 2.0], [157586.0785, 281847.7155, 1565521.471, 638240.0714, 662150.9056, 422114.3683, 1004416.992, 2.0, 2.0, 3.0, 1107753.069, 3730081.05, 2563258.778, 1505960.976, 3090159.735, 2368890.371, 1968232.774, 2.0, 2.0, 2.0], [163202.669, 255260.5647, 1745871.945, 562776.848, 624626.7385, 339327.2242, 979532.8968, 3.0, 3.0, 2.0, 1226458.688, 4420591.254, 2447710.88, 1687392

In [43]:
res=[]
for test in test_set:
    test_torch=torch.tensor(test).unsqueeze(0).float()
    # new_model = new_model.float()
    batch=new_model.batch(test_torch)
    # print(batch)
    linnear1_out=new_model.linear1(batch)
    rel=new_model.relu(linnear1_out)
    # print(rel)
    linnear2_out=new_model.linear2(rel)
    # print(linnear2_out)
    pool=new_model.pool(linnear2_out)
    res.append(pool.detach().numpy())

In [44]:
print(res)

[array([[-12.627738, -50.697975, -32.124874,  18.698046, -60.064312]],
      dtype=float32), array([[-16.092545, -52.35233 , -19.79373 ,  23.5007  , -67.456726]],
      dtype=float32), array([[-18.366362, -35.704823,  -6.338453,  16.62911 , -45.727192]],
      dtype=float32), array([[-14.391364, -40.589897, -15.618772,  19.866062, -49.212856]],
      dtype=float32), array([[-14.441149, -39.385292, -12.678906,  16.91406 , -53.047955]],
      dtype=float32), array([[-16.93603 , -35.55274 , -13.285197,  16.744041, -43.432774]],
      dtype=float32), array([[-16.108622, -43.77421 , -11.723731,  19.704842, -55.6875  ]],
      dtype=float32), array([[-13.836208, -29.638752, -12.637738,  14.13059 , -38.010456]],
      dtype=float32), array([[-14.516992, -36.130314, -16.371695,  16.90091 , -43.862827]],
      dtype=float32), array([[-15.803946, -42.99101 , -10.126115,  18.660875, -54.856144]],
      dtype=float32), array([[-17.685648, -41.394115, -20.085363,  17.682333, -49.48779 ]],
      dty

In [118]:
batch_c = np.array([1.829391,0.506226,0.007547,2.598037,-0.663109,1.934073,0.310004,-0.381942,2.902544,-0.272655,0.194586,2.603212,1.712726,0.265237,3.417350,-0.475785,-0.156884,1.607342,-0.748556,2.079414])
batch_python = batch.detach().numpy()
print(batch_c-batch_python)

linear1_c = np.array([0.000000,5.344080,0.000000,3.055317,5.794571,10.467493,11.741824,3.638603,0.000000,0.610046,0.000000,0.502744,0.000000,2.675269,0.000000,2.099277,3.195400,6.882000,0.000000,10.711428,0.000000,0.000000,7.553976,1.234577,0.000000,0.000000,0.000000,0.000000,11.221336,0.000000,0.000000,0.000000,0.000000,0.000000,4.986940,8.595300,0.000000,0.000000,0.000000,2.504881,3.432056,0.828595,0.000000,0.000000,0.000000,6.978451,0.178634,6.207349,0.000000,0.000000,4.537116,9.299417,0.000000,1.076422,1.226855,0.000000,3.596836,1.953598,0.000000,0.000000,0.000000,4.002781,0.000000,0.803060])
linear1_python = rel.detach().numpy()
print(linear1_c-linear1_python)

[[-0.20861507 -0.10559988 -0.17926719  0.06833338 -0.04562586 -0.04503426
  -0.05946803 -0.11486992  0.02110884  0.04083134 -0.23179964 -0.07472646
   0.03722847  0.033479   -0.17544466 -0.41470156 -0.05942681 -0.21915961
  -0.03054438 -0.2028612 ]]
[[ 0.         -0.00489041  0.          0.1233433  -0.31148664 -0.1310969
   0.14447102 -0.43757221  0.         -0.14064684  0.          0.33114899
   0.          0.05122887  0.          0.25522445  0.26268244 -0.83290335
   0.         -0.40596359  0.          0.          0.38300937  0.51853895
   0.          0.          0.          0.         -0.12391508  0.
   0.          0.          0.          0.          0.47256384  0.09320287
   0.          0.          0.          0.48810611  0.36547356 -0.3770682
   0.          0.          0.         -0.48300625 -0.1641603  -0.25678951
   0.          0.         -0.69035631 -0.77098228  0.          0.50313356
   0.0345585   0.          0.42752162 -0.25508478  0.          0.
   0.          0.12841138  0

In [ ]:
class LayerData:
    def __init__(self, data):
        self.data = data
        self.size = len(data)

# 假设以下变量已经定义和初始化
batch_normal_running_mean = new_model.batch.running_mean  # 运行时均值
batch_normal_running_var = new_model.batch.running_var  # 运行时方差
batch_normal_weight = [weight_0, weight_1, weight_2, ...]  # 权重
batch_normal_bias = new_model.batch.bias  # 偏差
EPS = 1e-5  # 避免除以零的小值

def batch_normal1d(input_data):
    for i in range(input_data.size):
        input_data.data[i] = (
            (input_data.data[i] - batch_normal_running_mean[i]) /
            (batch_normal_running_var[i] + EPS) ** 0.5
        ) * batch_normal_weight[i] + batch_normal_bias[i]
    return "NET_NORMAL"  # 返回NET_NORMAL作为标志

# 示例用法
input_data = LayerData([data_0, data_1, data_2, ...])  # 假设输入数据已经准备好
result = batch_normal1d(input_data)
print(input_data.data)  # 输出处理后的数据

In [155]:
class SCModel(nn.Module):
    def __init__(self, in_channels=32, num_classes=4):
        super(SCModel, self).__init__()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(20,64)
        self.batch = nn.BatchNorm1d(64)
        self.pool = nn.MaxPool1d(10)
        self.linear2 = nn.Linear(64, 128)
        self.linear3 = nn.Linear(128, 44)
        self.soft = nn.Softmax(dim=1)
    

    def forward(self, x):
        x = self.linear1(x) 
        x = self.relu(x)
        x = self.batch(x)   
        x = self.linear2(x)
        x = self.pool(x) 
        # x = self.relu(x)
        # x=self.linear3(x)
        x = self.soft(x)
        # print(x)
        return x

In [110]:
lo=torch.load('best_model_final_5class_95928.pth')

In [112]:
x_train_torch=torch.from_numpy(X_train)
y_train_torch=torch.from_numpy(y_train)

train_dataset = TensorDataset(x_train_torch,y_train_torch)
split_tra=int(len(y_train))
train_loader=DataLoader(dataset=train_dataset,batch_size=16,shuffle=False,drop_last=True)
# 测试集上评估模型性能
lo.eval()
num_correct=0
num_samples=0
with torch.no_grad():
    for x,y in train_loader:
        # print(x)
        x=x.to(device)
        y=y.to(device)
        preds=lo(x)
        predictions=preds.max(1).indices
        # print(predictions,y)
        num_correct+=(predictions==y).sum()
        num_samples+=predictions.size(0)
    acc=(num_correct/num_samples).item()
    print(num_correct,num_samples)


tensor(3067) 3600


In [292]:
lo.batch.running_var

tensor([6.5363e+11, 1.7904e+12, 4.4119e+11, 1.2119e+12, 5.5921e+11, 6.1156e+11,
        1.1487e+12, 3.8972e+11, 6.4467e+11, 3.9562e+11, 2.3605e+12, 1.8622e+12,
        1.0049e+12, 6.6055e+11, 2.9501e+11, 4.3222e+11, 4.5101e+11, 1.2403e+12,
        3.9657e+12, 7.4823e+11, 1.3901e+12, 1.7263e+12, 1.3897e+12, 2.8831e+11,
        8.3141e+11, 1.0059e+12, 9.8961e+11, 1.1152e+12, 7.0736e+11, 8.9069e+11,
        4.0339e+11, 2.9009e+11, 2.9648e+11, 6.5426e+11, 8.1387e+11, 3.5724e+11,
        1.4686e+11, 4.4935e+11, 7.0746e+11, 7.1295e+11, 1.5109e+12, 2.7495e+11,
        6.3568e+11, 3.9809e+11, 2.2144e+12, 2.0150e+11, 7.9950e+11, 5.2580e+11,
        4.4747e+11, 9.8991e+11, 3.5636e+11, 2.3902e+12, 2.5955e+12, 9.1168e+11,
        4.5450e+11, 6.6289e+11, 2.1233e+12, 4.2171e+11, 2.0481e+12, 4.9990e+11,
        3.1689e+11, 1.3274e+12, 3.3693e+12, 1.9151e+12, 9.7625e+11, 7.0196e+11,
        5.6760e+11, 5.4699e+12, 7.5834e+11, 1.2814e+12, 3.6344e+12, 1.0019e+12,
        4.0979e+11, 9.9092e+11, 1.7114e+

In [270]:
# compare
test=[1057137.047296, 363753.23750399996, 1886221.580416, 563634.4466451614, 1403906.8255483871, 718067.5610967746, 3302.0, 2087.0, 4525.0, 23.25, 24.25, 23.5, 5.0, 5.0, 5.0, 3655135.0704639996, 42998.1664, 1241013.519104, 249326.28483870966, 2086669.9657419352, 142252.04096774195, 6054.0, 796.0, 3662.0, 24.2, 48.0, 23.5, 6.0, 3.0, 5.0]
test_torch=torch.tensor(test)
linnear3_out=lo.linear1(test_torch.double())
rel=lo.relu(linnear3_out)
print(rel)
batch=lo.batch(rel.unsqueeze(0))
print(batch)
linnear4_out=lo.linear2(batch)
print(linnear4_out)
pool=lo.pool(linnear4_out)
print(pool)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 20x128)

In [684]:
import math

In [59]:
print(lo.batch.eps)
print(lo.batch.weight)
print(lo.batch.bias)

1e-05
Parameter containing:
tensor([ 4.1409,  3.9793,  6.7132,  6.3897,  3.3582,  6.1925,  9.4022,  4.0726,
         3.4271,  8.5347,  2.9962,  5.2588,  2.1316,  0.8751,  8.3284,  0.4437,
         8.2700,  6.3592, -0.6558,  2.3517,  1.3832, 10.3203,  7.0237,  1.8826,
         1.4922,  7.6236,  7.8416,  6.9612,  8.7300, 17.7785,  3.1899,  2.9794],
       dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.6822,  2.3519,  0.6459, -0.2304,  1.3256,  1.3849, -0.9953,  1.1506,
         0.4308,  0.7978,  1.0627,  1.7650, -0.0499,  0.6096,  0.8985,  0.1148,
        -0.5946,  0.6498,  0.0310,  0.4694, -0.4588,  1.7881,  0.5306,  0.3917,
         0.0027, -0.6870,  1.7942,  1.6242, -0.9404,  1.7163,  0.0748, -0.4968],
       dtype=torch.float64, requires_grad=True)


In [60]:
# mea = a.mean(dim=0)
# var = a.var(dim=0, unbiased=False)
# print(mea)
# print(var)
lo.batch.running_var

tensor([2.4703e-323,  1.2103e+10,  5.1708e+10,  1.2235e+03, 2.4703e-323,
         5.4104e+11,  1.2913e+12,  1.1412e+03,  1.3178e+04,  4.5649e+10,
         1.5845e+04,  2.1060e+12, 2.4703e-323,  3.4983e+08,  2.1644e+12,
         7.4315e+08,  2.6089e+10,  4.5677e+13,  1.5625e+04,  1.5307e+02,
        2.4703e-323,  1.4147e+13, 2.4703e-323, 2.4703e-323, 2.4703e-323,
         3.2124e+11,  1.0636e+11,  1.6354e+11,  1.8677e+11,  3.5847e+13,
        2.4703e-323,  1.3092e+04], dtype=torch.float64)

In [62]:
for i in lo.batch.running_mean:
    print(str(float(i))+',',end='')

2.5e-323,6210.444954292855,206508.75781464158,0.03528871173065771,2.5e-323,371417.030711922,582724.2854326313,7.2460104417197035,26.783799364272095,180948.82703214805,32.34233011534481,895742.0954127107,2.5e-323,885.359881791216,616722.238558261,1506.4684256137616,77003.72681670953,3099322.0205624867,5.052481065644358,2.607770897356141,2.5e-323,2576802.218328275,2.5e-323,2.5e-323,2.5e-323,342297.7120508508,108363.0680947516,276723.279505874,216336.915493771,5635237.071581095,2.5e-323,18.560829184005968,

In [63]:
for i in lo.batch.running_var:
    print(str(float(i))+',',end='')


2.5e-323,12103385541.984762,51707935527.91116,1223.4547961337837,2.5e-323,541039144409.66895,1291307784947.4827,1141.2347632397457,13178.441565919551,45648893998.58545,15845.058602244351,2106041985582.3394,2.5e-323,349831171.7461941,2164407632738.7273,743149714.0491723,26089319951.54715,45676686733345.89,15624.661921218336,153.07346873266962,2.5e-323,14146595741695.098,2.5e-323,2.5e-323,2.5e-323,321239721740.20483,106358491810.26958,163543047479.4659,186767599513.6804,35847043925389.44,2.5e-323,13091.972944081457,

In [712]:
torch.var(a.double(), unbiased=False)

tensor(6.4648, dtype=torch.float64)

In [722]:
var = np.var(l)
e=np.mean(l)
for i in range(32):
    
    print(((l[i]-lo.batch3.running_mean[i])/math.sqrt(lo.batch3.running_var[i]+1e-05))*lo.batch3.weight[i]+(lo.batch3.bias[i]))

tensor(-1.2443, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.1164, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.6433, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.5372, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.2080, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.1600, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.4909, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.5075, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(4464.4428, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.5872, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.0013, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(948.9407, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.4199, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(0.0070, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.4757, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(0.3352, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-1.5064, dtype=t

In [77]:
c=0
for i in lo.linear1.weight:
    for j in i:
        c+=1
        print(float(j),",",end="")
        if c%10==0:
            print()
print(c)

-0.973134380492561 ,-1.467509375193316 ,-1.3391505347306052 ,-1.8864621006111137 ,-0.7180657808974834 ,-2.0285279283494906 ,1.320949569650306 ,-0.7182207416025633 ,0.12695301238026752 ,3.3829044869752125 ,
2.910489796767462 ,2.875370697156869 ,2.884483940918128 ,1.8507063038043476 ,2.3432701331557912 ,-0.85881819398129 ,-2.1894246469614744 ,-0.6699958407912608 ,-2.677774675723008 ,-1.571126677790113 ,
0.19586895989247644 ,0.5824781878102416 ,-0.22873700766791338 ,-0.30281634012629793 ,1.4642974864531364 ,1.1972384744790083 ,2.504803492275699 ,0.6026436124849311 ,0.7883569965381814 ,2.6045854245885076 ,
-1.190664580275883 ,0.4724838618903984 ,0.21787686942706758 ,-1.9157398125938905 ,-0.4221141600714192 ,-1.6441496399170181 ,4.476586331794474 ,3.702536437355796 ,11.27873181665082 ,6.287360084231576 ,
6.1529186299301495 ,7.819587293740899 ,2.9525269339949722 ,4.824610289883156 ,2.0237720605521514 ,-0.006390313492384364 ,0.6439149791301998 ,-0.7707690341573837 ,-1.1881597725434832 ,-1.806

In [74]:
c=0
for i in lo.linear1.bias:
    c+=1
    print(float(i),",",end="")
    if c%10==0:
        print()

3.126721268314084 ,-0.030176150458668217 ,0.28618923595585183 ,0.16375961899361222 ,0.11351134111885466 ,-4.180024413197137 ,17.99347619535329 ,12.08707961345575 ,3.135463808664364 ,-4.214224668548823 ,
15.065665558494313 ,4.114774738788555 ,-1.4592817778006912 ,1.2292655416782692 ,-0.47297017705910255 ,5.49264821452507 ,15.568559906710796 ,3.4434995893897082 ,-1.8672406612034023 ,10.929414812816033 ,
-0.08771840586932654 ,0.13678224773463998 ,0.6620800956253062 ,-0.3673658571393724 ,0.30305141857706225 ,7.463332828942716 ,-5.851516141073232 ,15.15297467287128 ,28.394268108123505 ,0.9878000000128696 ,
1.8635512014730744 ,0.9463436452003546 ,

In [78]:
c=0
for i in lo.linear2.weight:
    for j in i:
        c+=1
        print(float(j),",",end="")
        if c%10==0:
            print()
print(c)

0.46599381112999283 ,0.726188010634483 ,0.09040103421366423 ,-1.2447623295979708 ,-0.8051896299948762 ,0.4712283006734343 ,1.033805818515886 ,0.2738493139758625 ,1.68680493921264 ,0.6022302302318581 ,
1.2763600810768128 ,-0.19935059718386292 ,1.502745755934523 ,-1.453627159531377 ,0.698119291857358 ,0.9749178350082374 ,0.970415407238323 ,1.305155676581974 ,-0.8347081759253255 ,-0.3036247887855288 ,
0.07342224339122996 ,-0.6309163343681048 ,1.4481154138411592 ,-0.6852507466812773 ,0.9595381972429516 ,0.7289830941877316 ,0.30801983667300115 ,0.006376164575942437 ,1.1846298975107854 ,0.8426653613160102 ,
0.21975764466095546 ,0.5805270030169555 ,4.1850067482372335 ,-0.6990016935157793 ,-5.090917137318523 ,2.1644374349877573 ,-4.751391214582182 ,-3.373296314825337 ,0.28374980846948816 ,-1.073531289494948 ,
2.143445724090584 ,-4.1044487000087395 ,1.5144641119901872 ,-5.847023162324953 ,5.293598554818004 ,-2.2632903608951755 ,0.14650279115314369 ,11.125211220710698 ,-1.1909571620402672 ,0.101

In [76]:
c=0
for i in lo.linear2.bias:
    c+=1
    print(float(i),",",end="")
    if c%10==0:
        print()

-2.47492603857373 ,-6.749053706893601 ,-4.238983078204463 ,-2.4302980216526118 ,-2.364214822978202 ,-1.331756706421174 ,-2.4288435641108284 ,-2.3884476454930996 ,-1.6308928620949599 ,-2.2544073362306185 ,
-2.1752405308959193 ,-3.1388041373217366 ,-1.5044671812283792 ,-1.3100587122915353 ,-0.4861557760958073 ,-1.4362800961268907 ,-5.256052246447474 ,-2.38963306916847 ,-2.1449229644819914 ,-5.187916023829814 ,
-1.6164937057227928 ,0.46785765329525786 ,-1.627546931526877 ,1.4716725388323288 ,

In [8]:
torch.onnx.export(lo,  torch.randn((1, 30)).double(),  "model2.onnx", export_params=True, opset_version=10,  do_constant_folding=True, input_names=['input'],  output_names=['output'])


In [119]:
class SCModel_MLP(nn.Module):
    def __init__(self, in_channels=32, num_classes=5):
        super(SCModel_MLP, self).__init__()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(20,32)
        self.batch = nn.BatchNorm1d(32)
        # self.pool = nn.MaxPool1d(2)
        self.linear2 = nn.Linear(32, num_classes)
        self.soft = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.linear1(x) 
        x = self.batch(x)  
        x = self.relu(x) 
        x = self.linear2(x)
        # x = self.pool(x) 
        # x = self.relu(x)
        # x=self.linear3(x)
        x = self.soft(x)
        # print(x)
        return x

In [120]:
model=SCModel_MLP()
model=model.to(device)
model._modules
model.double()

SCModel_MLP(
  (relu): ReLU()
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (batch): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=32, out_features=5, bias=True)
  (soft): Softmax(dim=1)
)

In [115]:
temp=3
# hard loss
hard_loss=nn.CrossEntropyLoss()
alpha=0.7

# soft_loss
soft_loss=nn.KLDivLoss(reduction="batchmean")
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)

In [ ]:
# 蒸馏
best_acc=0.0
epochs=300
for epoch in range(epochs):
    lr_s=3e-4
    optimizer=torch.optim.Adamax(model.parameters(),lr=lr_s)
    epochs=100
    acc_list=[]
    loss_list=[]
    best_model=model
    # kinds=2
    # best_teacher_matrix=torch.zeros(kinds, kinds)
    if (epoch%5==0):
        lr_s=lr_s-0.1*lr_s
        if(lr_s<1e-6):
            lr_s*=10
        # print("lr=",lr_s)
    optimizer=torch.optim.Adam(model.parameters(),lr=lr_s)
    model.train()
    #训练集上训练模型权重
    for data,targets in tqdm(train_loader):
        data=data.to(device)
        targets=targets.to(device)
        # 教师模型预测
        with torch.no_grad():
            teacher_preds=lo(data)
        # print(teacher_preds)
        # 学生模型预测
        student_preds=model(data)
        # 计算hard_loss
        # student_loss=hard_loss(student_preds,targets)
        student_loss=hard_loss(student_preds,targets.long())
        #计算蒸馏后的预测结果及soft_loss
        # print(student_preds.shape,teacher_preds.shape)
        ditillation_loss=soft_loss(
            F.softmax(student_preds/temp,dim=1),
            F.softmax(teacher_preds/temp,dim=1)
        )
        #将hard_loss和soft_loss加权求和
        # loss=alpha*student_loss-(1-alpha)*ditillation_loss
        # print(loss,student_loss,ditillation_loss)
        loss=student_loss
        #反向传播，权重优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # break
    # 测试集上评估模型性能
    model.eval()
    num_correct=0
    num_samples=0
    num_correct_train=0
    num_samples_train=0
    with torch.no_grad():
        for x,y in test_loader:
            x=x.to(device)
            y=y.to(device)
            preds=model(x)
            predictions=preds.max(1).indices
            num_correct+=(predictions==y).sum()
            num_samples+=predictions.size(0)
        acc=(num_correct/num_samples).item()
        print(num_correct,num_samples)
        
    print("\b Epoch:{}\t Accuracy:{:.4f}".format(epoch+1,acc))
    if(acc>best_acc):
        best_acc=acc
        best_dist_model=copy.deepcopy(model)
    with torch.no_grad():
        for x,y in train_loader:
            x=x.to(device)
            y=y.to(device)
            
            preds=model(x)
            predictions=preds.max(1).indices
            num_correct_train+=(predictions==y).sum()
            num_samples_train+=predictions.size(0)
        acc=(num_correct_train/num_samples_train).item()
        print(num_correct_train,num_samples_train)
        
    model.train()
    print("Epoch:in train set {}\t Accuracy:{:.4f}".format(epoch+1,acc))


In [ ]:
best_acc

In [393]:
x_train_torch=torch.from_numpy(X_train)
y_train_torch=torch.from_numpy(y_train)

train_dataset = TensorDataset(x_train_torch,y_train_torch)
split_tra=int(len(y_train))
train_loader=DataLoader(dataset=train_dataset,batch_size=16,shuffle=False,drop_last=True)
# 测试集上评估模型性能
lo.eval()
num_correct=0
num_samples=0
with torch.no_grad():
    for x,y in train_loader:
        # print(x)
        x=x.to(device)
        y=y.to(device)
        preds=best_dist_model(x)
        predictions=preds.max(1).indices
        print(predictions,y)
        num_correct+=(predictions==y).sum()
        num_samples+=predictions.size(0)
    acc=(num_correct/num_samples).item()
    print(num_correct,num_samples)


tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]) tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       dtype=torch.float64)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [394]:
3548/3600

0.9855555555555555